## Imports

In [12]:
import tensorflow as tf
import numpy as np
import os

## Dataset preparation

In [2]:
TRAINING_DATA_DIR = "data/training_data"

data_files = [f"{TRAINING_DATA_DIR}/{file}" for file in os.listdir(TRAINING_DATA_DIR)]
dataset = tf.data.TFRecordDataset(filenames=data_files,
                                  compression_type="GZIP",
                                  num_parallel_reads=4)

In [3]:
possible_results = ["0-1", "1-0", "1/2-1/2"]
result_to_onehot = tf.keras.layers.StringLookup(
    vocabulary=possible_results, output_mode="one_hot")

for result in possible_results:
    print(f"{result} -> {result_to_onehot(result)}")

0-1 -> [0. 1. 0. 0.]
1-0 -> [0. 0. 1. 0.]
1/2-1/2 -> [0. 0. 0. 1.]


In [4]:
MAX_SEQUENCE_LENGTH = 1024

pieces = ["p", "n", "b", "r", "q", "k"]
rank_names = ["1", "2", "3", "4", "5", "6", "7", "8"]
file_names = ["a", "b", "c", "d", "e", "f", "g", "h"]
squares = [f + r for r in rank_names for f in file_names]
promotions = ["-", "=n", "=b", "=r", "=q"]
possible_tokens = pieces + squares + promotions

moves_vectorizer = tf.keras.layers.TextVectorization(
    output_mode="int",
    vocabulary=possible_tokens,
    standardize=None,
    split="whitespace",
    output_sequence_length=MAX_SEQUENCE_LENGTH
)

print("Example conversion")
sample_game = "p e2 e4 - p a7 a5 - b f1 c4 - p c7 c5 - q d1 f3 - p e7 e5 - q f3 f7 -"
print("Original:", sample_game)
print("Vectorized:", moves_vectorizer(sample_game).numpy()[:40])
print()

for token in possible_tokens:
    print(f"{token} -> {moves_vectorizer(token)[0]}")

Example conversion
Original: p e2 e4 - p a7 a5 - b f1 c4 - p c7 c5 - q d1 f3 - p e7 e5 - q f3 f7 -
Vectorized: [ 2 20 36 72  2 56 40 72  4 13 34 72  2 58 42 72  6 11 29 72  2 60 44 72
  6 29 61 72  0  0  0  0  0  0  0  0  0  0  0  0]

p -> 2
n -> 3
b -> 4
r -> 5
q -> 6
k -> 7
a1 -> 8
b1 -> 9
c1 -> 10
d1 -> 11
e1 -> 12
f1 -> 13
g1 -> 14
h1 -> 15
a2 -> 16
b2 -> 17
c2 -> 18
d2 -> 19
e2 -> 20
f2 -> 21
g2 -> 22
h2 -> 23
a3 -> 24
b3 -> 25
c3 -> 26
d3 -> 27
e3 -> 28
f3 -> 29
g3 -> 30
h3 -> 31
a4 -> 32
b4 -> 33
c4 -> 34
d4 -> 35
e4 -> 36
f4 -> 37
g4 -> 38
h4 -> 39
a5 -> 40
b5 -> 41
c5 -> 42
d5 -> 43
e5 -> 44
f5 -> 45
g5 -> 46
h5 -> 47
a6 -> 48
b6 -> 49
c6 -> 50
d6 -> 51
e6 -> 52
f6 -> 53
g6 -> 54
h6 -> 55
a7 -> 56
b7 -> 57
c7 -> 58
d7 -> 59
e7 -> 60
f7 -> 61
g7 -> 62
h7 -> 63
a8 -> 64
b8 -> 65
c8 -> 66
d8 -> 67
e8 -> 68
f8 -> 69
g8 -> 70
h8 -> 71
- -> 72
=n -> 73
=b -> 74
=r -> 75
=q -> 76


In [5]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

feature_description = {
    "moves": tf.io.FixedLenFeature([], tf.string, default_value=''),
    "white_elo": tf.io.FixedLenFeature([], tf.int64, default_value=0),
    "black_elo": tf.io.FixedLenFeature([], tf.int64, default_value=0),
    "result": tf.io.FixedLenFeature([], tf.string, default_value=''),
}

def prepare_example(example_proto):
    example = tf.io.parse_example(example_proto, feature_description)
    result_embedding = result_to_onehot(example["result"])
    tokenized_moves = moves_vectorizer(example["moves"])
    return tokenized_moves, example["white_elo"], example["black_elo"], result_embedding

def make_batches(ds):
  return (
      ds
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(prepare_example, tf.data.AUTOTUNE)
      .prefetch(buffer_size=tf.data.AUTOTUNE))

ds = make_batches(dataset)

## InputEmbedding Layer

In [8]:
class InputEmbedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.embedding_layer = tf.keras.layers.Embedding(
            input_dim=vocab_size, output_dim=embedding_dim)

    def call(self, x):
        x = self.embedding_layer(x)
        x *= tf.math.sqrt(tf.cast(self.embedding_dim, tf.float32))
        return x

In [9]:
vocab_size = moves_vectorizer.vocabulary_size()
test_input_embedding = InputEmbedding(vocab_size=vocab_size, embedding_dim=5)
for moves, white_elo, black_elo, result in ds.take(1):
    print(test_input_embedding(moves))

tf.Tensor(
[[[ 0.08719932 -0.01575967  0.0407076   0.10681532 -0.02268509]
  [ 0.07930453 -0.06022277  0.05317946 -0.05750503  0.07510678]
  [-0.07729118  0.10778025  0.04365802  0.10368367  0.05111607]
  ...
  [ 0.03788096 -0.05493489  0.04061383 -0.10936422 -0.00659587]
  [ 0.03788096 -0.05493489  0.04061383 -0.10936422 -0.00659587]
  [ 0.03788096 -0.05493489  0.04061383 -0.10936422 -0.00659587]]

 [[ 0.08719932 -0.01575967  0.0407076   0.10681532 -0.02268509]
  [ 0.07930453 -0.06022277  0.05317946 -0.05750503  0.07510678]
  [-0.07729118  0.10778025  0.04365802  0.10368367  0.05111607]
  ...
  [ 0.03788096 -0.05493489  0.04061383 -0.10936422 -0.00659587]
  [ 0.03788096 -0.05493489  0.04061383 -0.10936422 -0.00659587]
  [ 0.03788096 -0.05493489  0.04061383 -0.10936422 -0.00659587]]

 [[ 0.08719932 -0.01575967  0.0407076   0.10681532 -0.02268509]
  [ 0.05297581 -0.05138646 -0.0228233  -0.06464192  0.00070788]
  [ 0.00218857  0.0010532   0.06446736  0.06184484 -0.01458992]
  ...
  [ 0.0

## PositionalEmbedding Layer

In [96]:
def positional_encoding(length, depth):
    half_depth = depth / 2
    
    positions = np.arange(length)[:, np.newaxis]
    depths = np.arange(half_depth)[np.newaxis, :] / half_depth
    
    angle_rates = 1 / (10000**depths)
    angle_rads = positions * angle_rates

    sin = np.sin(angle_rads)
    cos = np.cos(angle_rads)

    positional_encoding = np.dstack((sin, cos)).reshape(sin.shape[0],-1)[:,:depth]
    
    return positional_encoding

In [99]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, embedding_dim):
        super().__init__()
        self.positional_embedding = positional_encoding(
            length=sequence_length, depth=embedding_dim)
        print(self.positional_embedding.shape)
        
    def call(self, x):
        x += self.positional_embedding
        return x

In [100]:
length = 10
dim = 5
positional_embedding_test = PositionalEmbedding(length, dim)
x = np.zeros((length, dim))
positional_embedding_test(x)

(10, 5)


<tf.Tensor: shape=(10, 5), dtype=float32, numpy=
array([[ 0.0000000e+00,  1.0000000e+00,  0.0000000e+00,  1.0000000e+00,
         0.0000000e+00],
       [ 8.4147096e-01,  5.4030228e-01,  2.5116222e-02,  9.9968451e-01,
         6.3095731e-04],
       [ 9.0929741e-01, -4.1614684e-01,  5.0216600e-02,  9.9873835e-01,
         1.2619144e-03],
       [ 1.4112000e-01, -9.8999250e-01,  7.5285293e-02,  9.9716204e-01,
         1.8928709e-03],
       [-7.5680250e-01, -6.5364361e-01,  1.0030649e-01,  9.9495661e-01,
         2.5238267e-03],
       [-9.5892429e-01,  2.8366220e-01,  1.2526439e-01,  9.9212337e-01,
         3.1547814e-03],
       [-2.7941549e-01,  9.6017027e-01,  1.5014327e-01,  9.8866427e-01,
         3.7857350e-03],
       [ 6.5698659e-01,  7.5390226e-01,  1.7492741e-01,  9.8458135e-01,
         4.4166869e-03],
       [ 9.8935825e-01, -1.4550003e-01,  1.9960120e-01,  9.7987723e-01,
         5.0476375e-03],
       [ 4.1211849e-01, -9.1113025e-01,  2.2414905e-01,  9.7455490e-01,
      